In [71]:
from datetime import datetime
import time
import sys
import torch, torchvision
# print("Torch version = ", torch.__version__)

# print("TorchVision version = ", torchvision.__version__)
# from torch.utils.collect_env import main
# main()


import detectron2
# print("Detectron2 version = ", detectron2.__version__)
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os
import json
import cv2
import random
import matplotlib.pyplot as plt 

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog


In [72]:
# Simple method to show the actual size of the image 
def display_image_in_actual_size(im_path):

    dpi = 80
    im_data = plt.imread(im_path)
    height, width, depth = im_data.shape

    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    ax.axis('off')

    # Display the image.
    ax.imshow(im_data, cmap='gray')

    plt.show()

In [73]:
display_image_in_actual_size("/home/nawaf/Downloads/I4K.jpg")

In [46]:
cfg = get_cfg()

# You can download the model in your local machine and then use it without "model_zoo.get_config_file".
# And only using "cfg.merge_from_file(cfgfile)".
cfgfile = ""

# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
# mask_rcnn_R_50_FPN_3x.yaml is 170MB
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

# set threshold for this model
# Threshold is basically when the model going to predict if the box/segmentation is true prediction
# Ex. if the threshold is 0.5 the anything has IoU or AP (not sure with of these metrics going to use to use to predict) more than or equal 0.5
# will concider as true prediction.  
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  

# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

# Tell it to use the CPU
# cfg.MODEL.DEVICE = "cpu"

predictor = DefaultPredictor(cfg)

In [47]:
startT = datetime.now()

# Predict input image
outputs = predictor(im)

endT = datetime.now()

print("This is prediction time on GPU  ", endT - startT)

This is prediction time on GPU   0:00:01.129457


In [69]:
# predict the number of instance in image.
print("number of instances in the image =", outputs["instances"].pred_classes.numel()) 
# x0 = outputs["instances"].pred_classes.numel()


# # predict/show the boxes as tesor.
# print(outputs["instances"].pred_boxes) 
# x1 = outputs["instances"].pred_boxes


# predict/show the boxes as list.
print("Predicted boxes: ", outputs["instances"].get_fields()["pred_boxes"].tensor.tolist()) 
# xx3 = outputs["instances"].get_fields()["pred_boxes"].tensor.tolist()


# # predict/show the masks as tensor.
# print(outputs["instances"].pred_masks)
# print(outputs["instances"].pred_masks.tolist())
# x3 = outputs["instances"].pred_masks.tolist()
# xx1 = outputs["instances"].pred_masks


# The predicted instances
# print(outputs["instances"].pred_classes)
# x4 = outputs["instances"].pred_classes


# The predicted instances as list 
# print(outputs["instances"].get_fields()["pred_classes"].tolist())
# x5 = outputs["instances"].get_fields()["pred_classes"].tolist()


# The scores of predicted instances.
print("Scores of predicted classes: ", outputs['instances'].get_fields()["scores"].tolist())
# x6 = outputs['instances'].get_fields()["scores"].tolist()



# # We can't predict the keypoints, because we used mask_rcnn_R_50_FPN_3x. Please check model_zoo to find keypoints model. 
# # print(outputs["instances"].pred_keypoints)

Pred_classes = []

# Go through the predicted class. 
for data in outputs["instances"].pred_classes:
    num = data.item() # The number of the instance 
    Pred_classes.append(MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes[num])
print("Predicted classes:", Pred_classes)



number of instances in the image = 1
Predicted boxes:  [[1858.72509765625, 213.12550354003906, 3369.080078125, 2870.73974609375]]
Scores of predicted classes:  [0.9900224804878235]
Predicted classes: ['person']


In [49]:
print("The size of predicted mask as a list = ", sys.getsizeof(x3), "bytes")
print("The size of predicted mask as a tensor = ", sys.getsizeof(xx1), "bytes")
print("The size of predicted box as a list = ", sys.getsizeof(xx3), "bytes")
print("The size of predicted class as a list = ", sys.getsizeof(x5), "bytes")
print("The size of scores of predicted classes as a list = ", sys.getsizeof(x6), "bytes")

The size of predicted mask as a list =  80 bytes
The size of predicted mask as a tensor =  80 bytes
The size of predicted box as a list =  80 bytes
The size of predicted class as a list =  80 bytes
The size of scores of predicted classes as a list =  80 bytes


## Visualization

In [74]:

# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)

# Using 'draw_instance_predication' we can draw instance-level prediction and bounding box results on an image.
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

# Draw the image with instance prediction
plt.axis("off")
plt.figure(figsize=(100, 100))
plt.imshow(out.get_image()[:, :, ::-1])